# Drazin inverse

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import linalg as la

### Problem 1

In [6]:
def p_1(A, k, A_D):
    if (np.allclose((A @ A_D),
                    (A_D @ A),
                    rtol=1e-05)
        and np.allclose((np.linalg.matrix_power(A, k+1) @ A_D),
                        (np.linalg.matrix_power(A, k)),
                        rtol=1e-05)
        and np.allclose((A_D @ A @ A_D),
                        (A_D),
                        rtol=1e-05)):
        
        output = True
    else:
        output = False
    print(output)
    return


In [8]:
A = np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
A_D = np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])
k_A=1

B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
B_D = np.zeros((3,3))
k_B = 3

In [7]:
p_1(A, k_A, A_D)

True


In [9]:
p_1(B, k_B, B_D)

True


### Problem 2

In [24]:
def p_2(A, tol=0.0000000001):
    n, n = A.shape
    f = lambda x: abs(x) > tol
    Q_1, S, k_1 = la.schur(A, sort=f)
    g = lambda x: abs(x) < tol
    Q_2, T, k_2 = la.schur(A, sort=g)
    U = np.column_stack((S[:,:k_1], T[:, :n-k_1]))
    U_1 = np.linalg.inv(U)
    V = U_1 @ A @ U
    Z = np.zeros((n,n), dtype=float)
    if k_1 != 0:
        M_1 = np.linalg.inv(V[:k_1,:k_1])
        Z[:k_1, :k_1] = M_1
    return U @ Z @ U_1
    

In [26]:
p2_dinverse = p_2(A)
p2_dinverse

array([[  1.,  -3.,   9.,  81.],
       [  0.,   1.,  -3., -18.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   0.,   0.]])

### Problem 3

In [60]:
def p_3(Ad):
    n, n = Ad.shape
    D = np.sum(Ad, 0)
    Deg = np.diag(D)
    Lap = Deg - Ad
    
    R = np.zeros((n,n))
    I = np.eye(n)
    
    for i in range(n):
        for j in range(n):
            L_j = Lap.copy()
            L_j[:, j] = I[:, j]
            L_j_d = p_2(L_j)
            if j == i:
                R[i, j] = 0
            else:
                R[i, j] = L_j_d[i, i]
    return R


In [61]:
#Testing
F1 = np.array([[0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0]])
p_3(F1)


array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 0.]])

In [62]:
F2 = np.array([[0, 1], [1, 0]])
p_3(F2)

array([[0., 1.],
       [1., 0.]])

In [63]:
F3 = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
p_3(F3)

array([[0.        , 0.66666667, 0.66666667],
       [0.66666667, 0.        , 0.66666667],
       [0.66666667, 0.66666667, 0.        ]])

In [65]:
F4 = np.array([[0, 3], [3, 0]])
p_3(F4)

array([[0.        , 0.33333333],
       [0.33333333, 0.        ]])

In [66]:
F5 = np.array([[0,2],[2,0]])
p_3(F5)

array([[0. , 0.5],
       [0.5, 0. ]])

In [67]:
F6 = np.array([[0,4],[4,0]])
p_3(F6)

array([[0.  , 0.25],
       [0.25, 0.  ]])

### Problem 4 and 5

In [79]:
import pandas as pd

In [175]:
class LinkPredictor(object):
    
    def __init__(self, filename):
        data = pd.read_csv(filename, header=None)
        self.names = np.unique(np.concatenate((data[0], data[1])))
        
        n = len(self.names)
        self.Adj = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                self.Adj[i,j] = (data[0][((data[0]==self.names[i])
                                       & (data[1]==self.names[j]))].count() +
                                 data[1][((data[1]==self.names[i])
                                        & (data[0]==self.names[j]))].count())
        
        self.R = p_3(self.Adj)
        
    def predict_link(self, node):
        cmat = self.R * (1 - self.Adj) #zero out connected nodes
        if node == None:
            min_res = np.min(cmat[cmat>0])
            loc = np.where(cmat == min_res)
            Y = (self.names[loc[0]], self.names[loc[1]])
        elif type(node) == str:
            index = np.where(self.names == node)
            col = cmat[:,index]
            min_res = np.min(col[col > 0])
            loc = np.where(col == min_res)
            Y = (self.names[loc[0]])
        print(Y)
        return
        
    def add_link(self, node1, node2):
        loc1 = np.where(self.names==node1)
        loc2 = np.where(self.names==node2)
        self.Adj[loc1, loc2] = 1
        self.Adj[loc2, loc1] = 1
        return

In [153]:
data = pd.read_csv('social_network.csv', header=None)
names = np.unique(np.concatenate((data[0], data[1])))
#data[0][data[0] == names[23]].count()
names[1]

'Alan'

In [170]:
test_1 = LinkPredictor('social_network.csv')
test.predict_link(node=None)


(array(['Emily'], dtype=object), array(['Oliver'], dtype=object))

In [171]:
test.predict_link(node='Melanie')

array(['Carol'], dtype=object)

In [173]:
test.predict_link(node='Alan')

array(['Sonia'], dtype=object)

In [176]:
test = LinkPredictor('social_network.csv')
test.add_link('Sonia', 'Alan')
test.predict_link('Alan')

['Piers']
